In [73]:
import pandas as pd
import matplotlib.pyplot as plt

In [74]:
import os
import pandas as pd

# Path ke folder yang berisi file-file DataFrame
folder_path = '../comodity-price-prediction-penyisihan-arkavidia-9/Harga Bahan Pangan/train'

# Dictionary untuk menyimpan semua DataFrame dengan nama file sebagai key
df_dict = {}

# Loop melalui semua file dalam folder
for file_name in os.listdir(folder_path):
    # Pastikan file yang dibaca adalah file yang diinginkan (misalnya, CSV)
    if file_name.endswith('.csv'):
        # Buat path lengkap ke file
        file_path = os.path.join(folder_path, file_name)
        
        # Baca file dan simpan ke dictionary dengan nama file sebagai key
        df = pd.read_csv(file_path)
        df_dict[file_name] = df  # Gunakan nama file sebagai key


In [75]:
df_bawang_merah = df_dict['Bawang Merah.csv']
df_bawang_putih = df_dict['Bawang Putih Bonggol.csv']
df_beras_medium = df_dict['Beras Medium.csv']
df_beras_premium = df_dict['Beras Premium.csv']
df_cabai_merah_keriting = df_dict['Cabai Merah Keriting.csv']
df_cabai_rawit_merah = df_dict['Cabai Rawit Merah.csv']
df_daging_ayam = df_dict['Daging Ayam Ras.csv']
df_daging_sapi = df_dict['Daging Sapi Murni.csv']
df_gula = df_dict['Gula Konsumsi.csv']
df_minyak_goreng_curah = df_dict['Minyak Goreng Curah.csv']
df_minyak_goreng_kemasan = df_dict['Minyak Goreng Kemasan Sederhana.csv']
df_telur_ayam = df_dict['Telur Ayam Ras.csv']
df_tepung_terigu = df_dict['Tepung Terigu (Curah).csv']

In [76]:
import pandas as pd

# List semua dataframe
df_list = [
    df_bawang_merah, df_bawang_putih, df_beras_medium, df_beras_premium, df_cabai_merah_keriting,
    df_cabai_rawit_merah, df_daging_ayam, df_daging_sapi, df_gula, df_minyak_goreng_curah,
    df_minyak_goreng_kemasan, df_telur_ayam, df_tepung_terigu
]

# Loop setiap dataframe dan lakukan interpolasi
for i in range(len(df_list)):
    df = df_list[i]  # Ambil dataframe

    # Pisahkan kolom tanggal (asumsi ada kolom 'date' atau 'tanggal')
    date_cols = df.select_dtypes(include=['datetime64', 'object'])  # Deteksi kolom tanggal
    num_cols = df.select_dtypes(include=['number'])  # Ambil hanya kolom numerik

    # Interpolasi hanya untuk kolom numerik
    df_num_imputed = num_cols.interpolate(method='linear', limit_direction='both')

    # Gabungkan kembali kolom tanggal dengan hasil interpolasi
    df_list[i] = pd.concat([date_cols.reset_index(drop=True), df_num_imputed.reset_index(drop=True)], axis=1)


In [77]:
df_dict = {
    'Bawang Merah': df_list[0],
    'Bawang Putih Bonggol': df_list[1],
    'Beras Medium': df_list[2],
    'Beras Premium': df_list[3],
    'Cabai Merah Keriting': df_list[4],
    'Cabai Rawit Merah': df_list[5],
    'Daging Ayam Ras': df_list[6],
    'Daging Sapi Murni': df_list[7],
    'Gula Konsumsi': df_list[8],
    'Minyak Goreng Curah': df_list[9],
    'Minyak Goreng Kemasan Sederhana': df_list[10],
    'Telur Ayam Ras': df_list[11],
    'Tepung Terigu (Curah)': df_list[12]
}


In [78]:
df_usd= pd.read_csv('usd_idr_clean.csv')

In [79]:
df_usd = df_usd.drop(columns='Volume')

df_usd

,Date,Adj Close,Close,High,Low,Open
0,2022-01-01,14215.000000,14215.000000,14301.299805,14207.500000,14246.000000
1,2022-01-02,14215.000000,14215.000000,14301.299805,14207.500000,14246.000000
2,2022-01-03,14215.000000,14215.000000,14301.299805,14207.500000,14246.000000
3,2022-01-04,14283.599609,14283.599609,14383.299805,14259.000000,14303.700195
4,2022-01-05,14385.500000,14385.500000,14398.500000,14336.700195,14346.400391
...,...,...,...,...,...,...
999,2024-09-26,15201.500000,15201.500000,15204.000000,14975.700195,15201.500000
1000,2024-09-27,15070.000000,15070.000000,15161.700195,15019.299805,15070.000000
1001,2024-09-28,15086.000000,15086.000000,15173.566732,14991.500000,15086.000000
1002,2024-09-29,15102.000000,15102.000000,15185.433268,14963.700195,15102.000000


In [80]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error

# Parameter
seq_length = 30  # Panjang sequence
learning_rate = 0.001
epochs = 100
# Rentang waktu prediksi
predict_start = '2024-10-01'
predict_end = '2024-12-31'
predict_periods = (pd.to_datetime(predict_end) - pd.to_datetime(predict_start)).days + 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [81]:

class MultivariateLSTM(nn.Module):
    def __init__(self, input_size=5, hidden_size=64, num_layers=3, output_size=1):
        super(MultivariateLSTM, self).__init__()

        # LSTM dengan bidirectional=True agar melihat pola dua arah
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                            batch_first=True, dropout=0.3, bidirectional=True)

        # Batch Normalization agar training lebih stabil
        self.batch_norm = nn.BatchNorm1d(2 * hidden_size)  # Karena bidirectional, hidden_size jadi 2x

        # Fully Connected Layer dengan Activation Function
        self.fc = nn.Linear(2 * hidden_size, output_size)
        self.relu = nn.ReLU()  # Activation agar output lebih fleksibel

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[:, -1, :]  # Ambil output terakhir
        lstm_out = self.batch_norm(lstm_out)  # Normalisasi
        return self.fc(self.relu(lstm_out))  # Lewat activation dulu sebelum FC layer

In [82]:
# Direktori penyimpanan model terbaik
if not os.path.exists("best_models"): os.makedirs("best_models")

# Menyimpan hasil prediksi dan MAPE terbaik
all_predictions = []
mape_results = {}

In [83]:
for komoditas, df_komoditas in df_dict.items():
    print(f"\n=== Memproses {komoditas} ===")

    df_komoditas['Date'] = pd.to_datetime(df_komoditas['Date'])
    df_usd['Date'] = pd.to_datetime(df_usd['Date'])  # Data USD/IDR

    # Gabungkan dengan data USD/IDR berdasarkan tanggal
    df_merged = df_komoditas.merge(df_usd, on='Date', how='left')
    
    for provinsi in df_komoditas.columns[1:]:
        print(f"  → Prediksi untuk {provinsi}...")

        df_prov = df_merged[['Date', provinsi, 'Adj Close', 'Close', 'High', 'Low', 'Open']].rename(
            columns={'Date': 'ds', provinsi: 'y'}
        )
        
        # Normalisasi data
        scaler = MinMaxScaler()
        df_scaled = scaler.fit_transform(df_prov[['y', 'Adj Close', 'Close', 'High', 'Low', 'Open']])
        
        X, y = [], []
        for i in range(len(df_scaled) - seq_length):
            X.append(df_scaled[i:i+seq_length, 1:])  # Semua fitur kecuali target
            y.append(df_scaled[i+seq_length, 0])  # Target harga komoditas
        
        X, y = np.array(X), np.array(y)
        X = torch.tensor(X, dtype=torch.float32).to(device)
        y = torch.tensor(y, dtype=torch.float32).unsqueeze(-1).to(device)
        
        # Pisahkan training & testing
        train_size = int(len(X) * 0.8)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]
        
        # Bangun model LSTM
        model = MultivariateLSTM(input_size=5).to(device)  # 5 fitur dari USD/IDR
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5, verbose=True)
        
        # Menyimpan model terbaik berdasarkan validation loss
        best_val_loss = float("inf")
        best_mape = float("inf")
        best_model_path = f"best_models/{komoditas}_{provinsi}.pt"

        # Train model
        for epoch in range(epochs):
            model.train()
            optimizer.zero_grad()
            output = model(X_train)
            loss = criterion(output, y_train)
            loss.backward()
            optimizer.step()
            
            if (epoch + 1) % 10 == 0:
                model.eval()
                with torch.no_grad():
                    test_output = model(X_test)
                    val_loss = criterion(test_output, y_test).item()  # Hitung validation loss
                    
                    test_output_np = test_output.cpu().numpy()
                    y_test_np = y_test.cpu().numpy()

                    # Inverse transform hasil prediksi dan data aktual
                    test_output_np = scaler.inverse_transform(np.concatenate((test_output_np, np.zeros((len(test_output_np), 5))), axis=1))[:, 0]
                    y_test_np = scaler.inverse_transform(np.concatenate((y_test_np, np.zeros((len(y_test_np), 5))), axis=1))[:, 0]
                    
                    # Hitung MAPE
                    mape = mean_absolute_percentage_error(y_test_np, test_output_np)

                print(f"Epoch {epoch+1}/{epochs}, Training Loss: {loss.item():.6f}, Validation Loss: {val_loss:.6f}, MAPE: {mape:.6f}")

                scheduler.step(val_loss)  # Sesuaikan learning rate berdasarkan validation loss

                # Simpan model jika validation loss lebih kecil
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    best_mape = mape  # Simpan MAPE dari model terbaik
                    torch.save(model.state_dict(), best_model_path)
        
        # Simpan hasil terbaik untuk komoditas & provinsi ini
        mape_results[f"{komoditas}_{provinsi}"] = best_mape

        # Load model terbaik untuk prediksi masa depan
        best_model = MultivariateLSTM(input_size=5).to(device)
        best_model.load_state_dict(torch.load(best_model_path))
        best_model.eval()
        
        # Prediksi masa depan
        last_sequence = X[-1].unsqueeze(0)  # Data terakhir sebagai input
        future_preds = []

        for _ in range(predict_periods):
            with torch.no_grad():
                next_pred = best_model(last_sequence).item()  # Prediksi harga komoditas saja
            future_preds.append(next_pred)
            
            # Ambil nilai USD/IDR terakhir (5 fitur)
            last_usd_features = last_sequence[:, -1, 1:].cpu().numpy()

            # Gabungkan harga prediksi + USD/IDR
            next_pred = np.array([[next_pred]])
            next_input = np.concatenate((next_pred, last_usd_features), axis=1)  

            # Konversi ke tensor
            next_input = torch.tensor(next_input, dtype=torch.float32).unsqueeze(0).to(device)  
            last_sequence = torch.cat([last_sequence[:, 1:, :], next_input], dim=1) 

        # Konversi kembali ke skala asli (hanya untuk harga komoditas)
        future_preds = np.array(future_preds).reshape(-1, 1)
        future_preds = scaler.inverse_transform(np.concatenate((future_preds, np.zeros((len(future_preds), 5))), axis=1))[:, 0]

        # Simpan hasil prediksi
        for i, pred in enumerate(future_preds):
            date = pd.to_datetime(predict_start) + pd.Timedelta(days=i)
            all_predictions.append({
                'id': f"{komoditas}/{provinsi}/{date.strftime('%Y-%m-%d')}",
                'price': pred
            })

# Simpan hasil prediksi ke CSV
submission_df = pd.DataFrame(all_predictions)
submission_df.to_csv('submission_multivariate_lstm_mape_validation.csv', index=False)

# Hitung dan tampilkan rata-rata MAPE dari semua model terbaik
avg_mape = np.mean(list(mape_results.values()))
print(f"\nRata-rata MAPE dari model terbaik: {avg_mape:.6f}")



=== Memproses Bawang Merah ===
  → Prediksi untuk Aceh...
Epoch 10/100, Training Loss: 0.089073, Validation Loss: 0.282108, MAPE: 0.362123


/home/sirius/Documents/DS/myenv/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 20/100, Training Loss: 0.050904, Validation Loss: 0.257361, MAPE: 0.327371
Epoch 30/100, Training Loss: 0.041189, Validation Loss: 0.268156, MAPE: 0.339613
Epoch 40/100, Training Loss: 0.031543, Validation Loss: 0.259875, MAPE: 0.334414
Epoch 50/100, Training Loss: 0.030242, Validation Loss: 0.212859, MAPE: 0.283956
Epoch 60/100, Training Loss: 0.028335, Validation Loss: 0.172638, MAPE: 0.253355
Epoch 70/100, Training Loss: 0.027036, Validation Loss: 0.144185, MAPE: 0.238012
Epoch 80/100, Training Loss: 0.025429, Validation Loss: 0.140595, MAPE: 0.236615
Epoch 90/100, Training Loss: 0.026529, Validation Loss: 0.155990, MAPE: 0.245205
Epoch 100/100, Training Loss: 0.025776, Validation Loss: 0.186683, MAPE: 0.261632
  → Prediksi untuk Bali...


/tmp/ipykernel_66693/3786150980.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(best_model_path))
/home/sirius/Documents/DS/myenv/

Epoch 10/100, Training Loss: 0.054081, Validation Loss: 0.163400, MAPE: 0.379858
Epoch 20/100, Training Loss: 0.040850, Validation Loss: 0.170214, MAPE: 0.391931
Epoch 30/100, Training Loss: 0.035100, Validation Loss: 0.175937, MAPE: 0.402527
Epoch 40/100, Training Loss: 0.030314, Validation Loss: 0.176030, MAPE: 0.401191
Epoch 50/100, Training Loss: 0.029267, Validation Loss: 0.204515, MAPE: 0.446560
Epoch 60/100, Training Loss: 0.028078, Validation Loss: 0.206636, MAPE: 0.446010
Epoch 70/100, Training Loss: 0.026893, Validation Loss: 0.229577, MAPE: 0.476023
Epoch 80/100, Training Loss: 0.026891, Validation Loss: 0.256240, MAPE: 0.514331
Epoch 90/100, Training Loss: 0.025592, Validation Loss: 0.239409, MAPE: 0.489346
Epoch 100/100, Training Loss: 0.026378, Validation Loss: 0.238948, MAPE: 0.487099


...

/tmp/ipykernel_66693/3786150980.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(best_model_path))
/home/sirius/Documents/DS/myenv/

  → Prediksi untuk Papua...
Epoch 10/100, Training Loss: 0.063853, Validation Loss: 0.105235, MAPE: 0.151348
Epoch 20/100, Training Loss: 0.044089, Validation Loss: 0.129005, MAPE: 0.174092
Epoch 30/100, Training Loss: 0.039516, Validation Loss: 0.146843, MAPE: 0.189528
Epoch 40/100, Training Loss: 0.035432, Validation Loss: 0.183400, MAPE: 0.219431
Epoch 50/100, Training Loss: 0.026197, Validation Loss: 0.198976, MAPE: 0.233031
Epoch 60/100, Training Loss: 0.024325, Validation Loss: 0.159893, MAPE: 0.208473
Epoch 70/100, Training Loss: 0.021791, Validation Loss: 0.159545, MAPE: 0.205458
Epoch 80/100, Training Loss: 0.021530, Validation Loss: 0.184339, MAPE: 0.222856
Epoch 90/100, Training Loss: 0.020521, Validation Loss: 0.165378, MAPE: 0.205548
Epoch 100/100, Training Loss: 0.020647, Validation Loss: 0.148897, MAPE: 0.189324
  → Prediksi untuk Riau...


/tmp/ipykernel_66693/3786150980.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model.load_state_dict(torch.load(best_model_path))
/home/sirius/Documents/DS/myenv/

Epoch 10/100, Training Loss: 0.066384, Validation Loss: 0.290662, MAPE: 0.294750
Epoch 20/100, Training Loss: 0.051874, Validation Loss: 0.298100, MAPE: 0.300225
Epoch 30/100, Training Loss: 0.044328, Validation Loss: 0.300700, MAPE: 0.301274
Epoch 40/100, Training Loss: 0.042317, Validation Loss: 0.304540, MAPE: 0.303003


KeyboardInterrupt: 

In [39]:
# Hitung rata-rata MAPE
avg_mape = np.mean(list(mape_results.values()))
print(f"\nRata-rata MAPE keseluruhan: {avg_mape:.6f}")

print("\nPrediksi selesai dan disimpan. Model terbaik telah disimpan berdasarkan MAPE terendah!")



Rata-rata MAPE keseluruhan: 0.140867

Prediksi selesai dan disimpan. Model terbaik telah disimpan berdasarkan MAPE terendah!
